In [36]:

import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model


Image Extraction and Processing


In [2]:
img = cv2.imread("./images/img_3.jpg")
img = cv2.resize(img, (450, 450))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(img_gray, (5, 5), 1)
img_threshold = cv2.adaptiveThreshold(img_blur, 255, 1, 1, 11, 2)
output_path = os.path.join("images", "processed_image.jpg")
cv2.imwrite(output_path, img_threshold)

True

Contour Mapping


In [3]:
imgcontour = img.copy()
imgBigContour = img.copy()
contours, hierarchy = cv2.findContours(
    img_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(imgcontour, contours, -1, (0, 255, 0), 3)
cv2.imshow("Sudoku", imgcontour)
cv2.waitKey(0)
cv2.destroyAllWindows()

BiggestContour


In [4]:
def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest, max_area

In [5]:
def reorder(points):
    points = points.reshape((4, 2))
    points_new = np.zeros((4, 1, 2), np.int32)
    add = points.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] = points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]
    return points_new

In [6]:
biggest, maxArea = biggestContour(contours)
if biggest.size != 0:
    biggest = reorder(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [450, 0], [0, 450], [450, 450]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (450, 450))
    imgDetectedDigits = imgWarpColored.copy()
    imgWarpGray = cv2.cvtColor(imgWarpColored, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Sudoku", imgWarpColored)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
def splitBoxes(img):
    rows = np.vsplit(img, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

In [29]:
model = tf.keras.models.load_model('models/myModel.h5')

In [24]:
def getPredection(boxes, model):
    result = []
    for image in boxes:
        img = np.asarray(image)
        img = img[4:img.shape[0] - 4, 4:img.shape[1] - 4]
        img = cv2.resize(img, (28, 28))
        img = img / 255
        img = img.reshape(1, 28, 28, 1)
        predictions = model.predict(img)
        # classIndex = model.predict_classes(img)
        classindex = np.argmax(predictions, axis=-1)
        probabilityValue = np.amax(predictions)
        # print(classindex, probabilityValue)
        if probabilityValue > 0.8:
            result.append(classindex[0])
        else:
            result.append(0)
    return result

In [30]:
# def displayNumbers(img, numbers, color=(0, 255, 0)):
#     secW = int(img.shape[1]/9)
#     secH = int(img.shape[0]/9)
#     for x in range(0, 9):
#         for y in range(0, 9):
#             if numbers[(y*9)+x] != 0:
#                 cv2.putText(img, str(numbers[(y*9)+x]),
#                             (x*secW+int(secW/2)-10, int((y+0.8)*secH)
#                              ), cv2.FONT_HERSHEY_COMPLEX_SMALL,
#                             2, color, 2, cv2.LINE_AA)
#     return img

In [32]:
boxes = splitBoxes(imgWarpGray)
numbers = getPredection(boxes, model)
print(numbers)
# imgDetectedDigits = displayNumbers(imgDetectedDigits, numbers, color=(255, 0, 255))
numbers = np.asarray(numbers)
posArray = np.where(numbers > 0, 0, 1)
print(posArray)

1/1 [==============================] - 0s 20ms/step
[1, 0, 0, 0, 8, 0, 0, 0, 9, 0, 5, 0, 6, 0, 1, 0, 2, 0, 0, 0, 0, 5, 0, 3, 0, 0, 0, 0, 9, 4, 1, 0, 4, 8, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 5, 9, 0, 8, 4, 6, 0, 0, 0, 0, 7, 0, 5, 0, 0, 0, 0, 0, 0, 3, 0, 9, 0, 7, 0, 5, 0, 0, 0, 1, 0, 0, 0, 3]
[0 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 0 1 0
 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 1 0 1
 1 1 0 1 1 1 0]


In [47]:
def solve(bo):
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find
    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i
            if solve(bo):
                return True
            bo[row][col] = 0
    return False

def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False
    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False
    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3
    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False
    return True

def print_board(bo):
    for i in range(len(bo)):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - - - ")
        for j in range(len(bo[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")
            if j == 8:
                print(bo[i][j])
            else:
                print(str(bo[i][j]) + " ", end="")

def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col
    return None

In [49]:
board = np.array_split(numbers, 9)
print(board)
try:
    print("Solving...")
    solve(board)
except:
    pass
print(board)

[array([1, 0, 0, 0, 8, 0, 0, 0, 9], dtype=int64), array([0, 5, 0, 6, 0, 1, 0, 2, 0], dtype=int64), array([0, 0, 0, 5, 0, 3, 0, 0, 0], dtype=int64), array([0, 9, 4, 1, 0, 4, 8, 3, 0], dtype=int64), array([3, 0, 0, 0, 0, 0, 0, 0, 5], dtype=int64), array([0, 1, 5, 9, 0, 8, 4, 6, 0], dtype=int64), array([0, 0, 0, 7, 0, 5, 0, 0, 0], dtype=int64), array([0, 0, 0, 3, 0, 9, 0, 7, 0], dtype=int64), array([5, 0, 0, 0, 1, 0, 0, 0, 3], dtype=int64)]
Solving...
[array([1, 0, 0, 0, 8, 0, 0, 0, 9], dtype=int64), array([0, 5, 0, 6, 0, 1, 0, 2, 0], dtype=int64), array([0, 0, 0, 5, 0, 3, 0, 0, 0], dtype=int64), array([0, 9, 4, 1, 0, 4, 8, 3, 0], dtype=int64), array([3, 0, 0, 0, 0, 0, 0, 0, 5], dtype=int64), array([0, 1, 5, 9, 0, 8, 4, 6, 0], dtype=int64), array([0, 0, 0, 7, 0, 5, 0, 0, 0], dtype=int64), array([0, 0, 0, 3, 0, 9, 0, 7, 0], dtype=int64), array([5, 0, 0, 0, 1, 0, 0, 0, 3], dtype=int64)]
